# 📊 Визуализация данных и система оповещений

## Анализ трендов и автоматические уведомления

## 5. 📊 Визуализация данных

В этом разделе мы создадим комплексную визуализацию результатов мониторинга экосистемы.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

print("✅ Библиотеки для визуализации загружены!")

### 5.1 Генерация тестовых данных

In [ ]:
def generate_test_monitoring_data(days=30):
    """
    Генерация тестовых данных мониторинга для демонстрации
    
    Args:
        days: Количество дней мониторинга
        
    Returns:
        DataFrame с данными мониторинга
    """
    print(f"📊 Генерация тестовых данных за {days} дней...")
    
    # Создание временных меток
    start_date = datetime.now() - timedelta(days=days)
    dates = [start_date + timedelta(days=i) for i in range(days)]
    
    # Виды животных
    species = ['bird', 'deer', 'bear', 'fox', 'rabbit']
    
    data = []
    
    for date in dates:
        for spec in species:
            # Генерация случайных данных с трендами
            base_count = {
                'bird': 50,
                'deer': 15,
                'bear': 2,
                'fox': 5,
                'rabbit': 20
            }[spec]
            
            # Добавление сезонной вариации
            day_of_year = date.timetuple().tm_yday
            seasonal_factor = 1 + 0.3 * np.sin(2 * np.pi * day_of_year / 365)
            
            # Случайная вариация
            count = int(base_count * seasonal_factor * np.random.uniform(0.7, 1.3))
            count = max(0, count)
            
            data.append({
                'date': date,
                'species': spec,
                'count': count,
                'temperature': np.random.uniform(10, 25),
                'humidity': np.random.uniform(40, 80),
                'audio_activity': np.random.uniform(0.3, 0.9)
            })
    
    df = pd.DataFrame(data)
    print(f"✅ Сгенерировано {len(df)} записей")
    return df

# Генерация данных
monitoring_data = generate_test_monitoring_data(days=30)
monitoring_data.head(10)

### 5.2 Временные ряды активности животных

In [ ]:
# График временных рядов
fig = px.line(
    monitoring_data,
    x='date',
    y='count',
    color='species',
    title='Динамика популяций животных',
    labels={'count': 'Количество обнаружений', 'date': 'Дата', 'species': 'Вид'},
    template='plotly_white'
)

fig.update_layout(
    height=500,
    hovermode='x unified'
)

fig.show()

### 5.3 Распределение видов

In [ ]:
# Общая статистика по видам
species_totals = monitoring_data.groupby('species')['count'].sum().reset_index()
species_totals = species_totals.sort_values('count', ascending=False)

fig = px.bar(
    species_totals,
    x='species',
    y='count',
    title='Общее количество обнаружений по видам',
    labels={'count': 'Всего обнаружений', 'species': 'Вид'},
    color='count',
    color_continuous_scale='Viridis'
)

fig.update_layout(height=400)
fig.show()

### 5.4 Тепловая карта активности

In [ ]:
# Создание pivot table для тепловой карты
heatmap_data = monitoring_data.pivot_table(
    values='count',
    index='species',
    columns=monitoring_data['date'].dt.day_name(),
    aggfunc='mean'
)

# Упорядочивание дней недели
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data[[col for col in day_order if col in heatmap_data.columns]]

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='YlOrRd', cbar_kws={'label': 'Средн. кол-во'})
plt.title('Средняя активность животных по дням недели')
plt.xlabel('День недели')
plt.ylabel('Вид')
plt.tight_layout()
plt.show()

### 5.5 Корреляция с факторами окружающей среды

In [ ]:
# Анализ корреляции для каждого вида
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Температура vs активность
for species in monitoring_data['species'].unique():
    species_data = monitoring_data[monitoring_data['species'] == species]
    axes[0].scatter(species_data['temperature'], species_data['count'], alpha=0.5, label=species)

axes[0].set_xlabel('Температура (°C)')
axes[0].set_ylabel('Количество обнаружений')
axes[0].set_title('Влияние температуры на активность')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Влажность vs активность
for species in monitoring_data['species'].unique():
    species_data = monitoring_data[monitoring_data['species'] == species]
    axes[1].scatter(species_data['humidity'], species_data['count'], alpha=0.5, label=species)

axes[1].set_xlabel('Влажность (%)')
axes[1].set_ylabel('Количество обнаружений')
axes[1].set_title('Влияние влажности на активность')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 🚨 Система оповещений

Автоматическое обнаружение аномалий и критических событий в экосистеме.

In [ ]:
class AlertSystem:
    """
    Система оповещений для мониторинга экосистемы
    
    Функциональность:
    - Обнаружение аномалий в популяциях
    - Мониторинг редких видов
    - Детекция резких изменений
    - Формирование отчётов о тревогах
    """
    
    def __init__(self):
        self.alerts = []
        self.thresholds = {
            'rare_species': ['bear'],  # Редкие виды
            'min_count_threshold': 1,  # Минимальное количество для тревоги
            'max_count_threshold': 100,  # Максимальное количество
            'sudden_drop_pct': 0.5,  # 50% снижение
            'sudden_spike_factor': 2.0  # Увеличение в 2 раза
        }
    
    def check_rare_species(self, df):
        """
        Проверка обнаружения редких видов
        """
        alerts = []
        
        for species in self.thresholds['rare_species']:
            species_data = df[df['species'] == species]
            if len(species_data) > 0:
                total_count = species_data['count'].sum()
                if total_count > 0:
                    alerts.append({
                        'type': 'rare_species',
                        'severity': 'INFO',
                        'species': species,
                        'message': f"Обнаружен редкий вид: {species} ({total_count} раз)",
                        'timestamp': datetime.now()
                    })
        
        return alerts
    
    def check_population_anomalies(self, df):
        """
        Проверка аномалий в популяциях
        """
        alerts = []
        
        for species in df['species'].unique():
            species_data = df[df['species'] == species].copy()
            species_data = species_data.sort_values('date')
            
            counts = species_data['count'].values
            
            if len(counts) < 2:
                continue
            
            # Вычисление статистики
            mean_count = np.mean(counts)
            std_count = np.std(counts)
            recent_count = counts[-1]
            
            # Проверка на аномально низкое значение
            if mean_count > 0:
                if recent_count < mean_count * self.thresholds['sudden_drop_pct']:
                    alerts.append({
                        'type': 'population_drop',
                        'severity': 'WARNING',
                        'species': species,
                        'message': f"Резкое снижение популяции {species}: {recent_count} (норма: {mean_count:.1f})",
                        'timestamp': datetime.now()
                    })
            
            # Проверка на аномально высокое значение
            if recent_count > mean_count * self.thresholds['sudden_spike_factor']:
                alerts.append({
                    'type': 'population_spike',
                    'severity': 'WARNING',
                    'species': species,
                    'message': f"Необычно высокая активность {species}: {recent_count} (норма: {mean_count:.1f})",
                    'timestamp': datetime.now()
                })
        
        return alerts
    
    def check_biodiversity(self, df):
        """
        Проверка биоразнообразия
        """
        alerts = []
        
        unique_species = df['species'].nunique()
        
        if unique_species < 3:
            alerts.append({
                'type': 'low_biodiversity',
                'severity': 'WARNING',
                'species': None,
                'message': f"Низкое биоразнообразие: обнаружено только {unique_species} видов",
                'timestamp': datetime.now()
            })
        
        return alerts
    
    def analyze(self, df):
        """
        Комплексный анализ данных и генерация оповещений
        """
        print("🔍 Анализ данных мониторинга...\n")
        
        all_alerts = []
        
        # Проверка редких видов
        rare_alerts = self.check_rare_species(df)
        all_alerts.extend(rare_alerts)
        
        # Проверка аномалий популяций
        pop_alerts = self.check_population_anomalies(df)
        all_alerts.extend(pop_alerts)
        
        # Проверка биоразнообразия
        bio_alerts = self.check_biodiversity(df)
        all_alerts.extend(bio_alerts)
        
        self.alerts = all_alerts
        
        # Вывод оповещений
        if all_alerts:
            print(f"⚠️  Обнаружено {len(all_alerts)} оповещений:\n")
            for i, alert in enumerate(all_alerts, 1):
                severity_icon = {'INFO': 'ℹ️ ', 'WARNING': '⚠️ ', 'CRITICAL': '🚨'}[alert['severity']]
                print(f"{i}. {severity_icon} [{alert['severity']}] {alert['message']}")
        else:
            print("✅ Аномалий не обнаружено")
        
        return all_alerts
    
    def export_alerts(self, filepath='data/results/reports/alerts.csv'):
        """
        Экспорт оповещений в CSV
        """
        if not self.alerts:
            print("⚠️  Нет оповещений для экспорта")
            return
        
        df_alerts = pd.DataFrame(self.alerts)
        df_alerts.to_csv(filepath, index=False)
        print(f"✅ Оповещения сохранены в {filepath}")

print("✅ Класс AlertSystem создан!")

### 6.1 Запуск системы оповещений

In [ ]:
# Создание системы оповещений
alert_system = AlertSystem()

# Анализ данных
alerts = alert_system.analyze(monitoring_data)

# Экспорт оповещений
if alerts:
    alert_system.export_alerts()

## 7. 📈 Комплексный дашборд

In [ ]:
def create_comprehensive_dashboard(df, alerts):
    """
    Создание комплексного дашборда с ключевыми метриками
    """
    # Создание подграфиков
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Временные ряды', 'Распределение по видам', 
                       'Тепловая карта', 'Статистика'),
        specs=[[{'type': 'scatter'}, {'type': 'bar'}],
               [{'type': 'heatmap'}, {'type': 'table'}]]
    )
    
    # 1. Временные ряды
    for species in df['species'].unique():
        species_data = df[df['species'] == species]
        fig.add_trace(
            go.Scatter(
                x=species_data['date'],
                y=species_data['count'],
                name=species,
                mode='lines+markers'
            ),
            row=1, col=1
        )
    
    # 2. Распределение по видам
    species_totals = df.groupby('species')['count'].sum().reset_index()
    fig.add_trace(
        go.Bar(
            x=species_totals['species'],
            y=species_totals['count'],
            name='Всего'
        ),
        row=1, col=2
    )
    
    # 3. Тепловая карта (упрощённая)
    pivot_data = df.pivot_table(
        values='count',
        index='species',
        columns=df['date'].dt.date,
        aggfunc='sum'
    ).fillna(0)
    
    fig.add_trace(
        go.Heatmap(
            z=pivot_data.values,
            x=pivot_data.columns.astype(str),
            y=pivot_data.index,
            colorscale='Viridis'
        ),
        row=2, col=1
    )
    
    # 4. Таблица статистики
    stats = df.groupby('species')['count'].agg(['mean', 'std', 'min', 'max']).reset_index()
    stats.columns = ['Вид', 'Среднее', 'Станд. откл.', 'Мин', 'Макс']
    
    fig.add_trace(
        go.Table(
            header=dict(values=list(stats.columns),
                       fill_color='paleturquoise',
                       align='left'),
            cells=dict(values=[stats[col] for col in stats.columns],
                      fill_color='lavender',
                      align='left',
                      format=[None, '.1f', '.1f', 'd', 'd'])
        ),
        row=2, col=2
    )
    
    # Настройка layout
    fig.update_layout(
        height=800,
        title_text="Дашборд мониторинга экосистемы",
        showlegend=True
    )
    
    return fig

# Создание и отображение дашборда
dashboard = create_comprehensive_dashboard(monitoring_data, alerts)
dashboard.show()

## 8. 💾 Экспорт отчётов

In [ ]:
def export_comprehensive_report(df, alerts, output_dir='data/results/reports'):
    """
    Экспорт комплексного отчёта
    """
    from pathlib import Path
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # 1. Сохранение данных мониторинга
    data_file = f"{output_dir}/monitoring_data_{timestamp}.csv"
    df.to_csv(data_file, index=False)
    print(f"✅ Данные мониторинга: {data_file}")
    
    # 2. Сохранение статистики
    stats = df.groupby('species')['count'].agg(['count', 'mean', 'std', 'min', 'max'])
    stats_file = f"{output_dir}/statistics_{timestamp}.csv"
    stats.to_csv(stats_file)
    print(f"✅ Статистика: {stats_file}")
    
    # 3. Сохранение оповещений
    if alerts:
        alerts_df = pd.DataFrame(alerts)
        alerts_file = f"{output_dir}/alerts_{timestamp}.csv"
        alerts_df.to_csv(alerts_file, index=False)
        print(f"✅ Оповещения: {alerts_file}")
    
    # 4. Создание текстового отчёта
    report_file = f"{output_dir}/report_{timestamp}.txt"
    with open(report_file, 'w', encoding='utf-8') as f:
        f.write("="*60 + "\n")
        f.write("ОТЧЁТ МОНИТОРИНГА ЭКОСИСТЕМЫ\n")
        f.write("="*60 + "\n\n")
        f.write(f"Дата формирования: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Период мониторинга: {df['date'].min()} - {df['date'].max()}\n")
        f.write(f"Всего записей: {len(df)}\n")
        f.write(f"Уникальных видов: {df['species'].nunique()}\n\n")
        
        f.write("-"*60 + "\n")
        f.write("СТАТИСТИКА ПО ВИДАМ\n")
        f.write("-"*60 + "\n")
        f.write(stats.to_string())
        f.write("\n\n")
        
        if alerts:
            f.write("-"*60 + "\n")
            f.write(f"ОПОВЕЩЕНИЯ ({len(alerts)})\n")
            f.write("-"*60 + "\n")
            for i, alert in enumerate(alerts, 1):
                f.write(f"{i}. [{alert['severity']}] {alert['message']}\n")
    
    print(f"✅ Текстовый отчёт: {report_file}")
    print(f"\n📊 Все отчёты сохранены в {output_dir}")

# Экспорт отчётов
export_comprehensive_report(monitoring_data, alerts)

## 9. 📝 Выводы

### Результаты работы системы

1. **Видеоанализ (YOLOv8)**:
   - ✅ Автоматическое обнаружение животных
   - ✅ Подсчёт уникальных особей
   - ✅ Треккинг траекторий движения

2. **Аудиоанализ (YAMNet)**:
   - ✅ Классификация звуков природы
   - ✅ Распознавание голосов птиц
   - ✅ Спектральный анализ

3. **Визуализация**:
   - ✅ Временные ряды активности
   - ✅ Тепловые карты
   - ✅ Интерактивные дашборды

4. **Система оповещений**:
   - ✅ Детекция аномалий
   - ✅ Мониторинг редких видов
   - ✅ Автоматические уведомления

### Практическое применение

Данное решение может быть использовано:
- 🏞️ В национальных парках для мониторинга дикой природы
- 🔬 В научных исследованиях биоразнообразия
- 🌲 Для оценки влияния человека на экосистемы
- 📊 При принятии решений по охране природы

### Дальнейшее развитие

- 🚀 Интеграция с IoT сенсорами
- 📱 Мобильное приложение для полевых исследований
- 🤖 Улучшение точности моделей ML
- 🌐 Web-интерфейс для удалённого доступа
- 📧 Email/SMS/Telegram оповещения